In [1]:
import numpy as np

In [12]:
Lx, Ly, Lz = 1, 1, 1
nodes = 5
hx = Lx / (nodes - 1)
hy = Ly / (nodes - 1)
hz = Lz / (nodes - 1)


time_ticks = 20
tau = 0.0001

X = np.arange(nodes)[:, np.newaxis, np.newaxis, np.newaxis] / (nodes - 1) * Lx
Y = np.arange(nodes)[np.newaxis, :, np.newaxis, np.newaxis] / (nodes - 1) * Ly
Z = np.arange(nodes)[np.newaxis, np.newaxis, :, np.newaxis] / (nodes - 1) * Lz
T = np.arange(time_ticks)[np.newaxis, np.newaxis, np.newaxis, :] * tau

analytical = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly) * np.sin(np.pi * Z / Lz) * np.cos(3 * T) 

# analytical = np.sin(3 * np.pi * X / Lx) * np.sin( 3 * np.pi * Y / Ly) * np.sin(np.pi * Z / Lz) * np.cos(4 * (T + np.pi)) 


In [13]:
print(analytical[:,:,:,0])

[[[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]]

 [[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 3.536e-01 5.000e-01 3.536e-01 6.123e-17]
  [0.000e+00 5.000e-01 7.071e-01 5.000e-01 8.660e-17]
  [0.000e+00 3.536e-01 5.000e-01 3.536e-01 6.123e-17]
  [0.000e+00 6.123e-17 8.660e-17 6.123e-17 1.060e-32]]

 [[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 5.000e-01 7.071e-01 5.000e-01 8.660e-17]
  [0.000e+00 7.071e-01 1.000e+00 7.071e-01 1.225e-16]
  [0.000e+00 5.000e-01 7.071e-01 5.000e-01 8.660e-17]
  [0.000e+00 8.660e-17 1.225e-16 8.660e-17 1.500e-32]]

 [[0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00]
  [0.000e+00 3.536e-01 5.000e-01 3.536e-01 6.123e-17]
  [0.000e+00 5.000e-01 7.071e-01 5.000e-01 8.660e-17]
  [0.000e+00 3.536e-01

In [17]:
def laplasian(a):
    padded = np.pad(a, 1, 'constant')
    out = (padded[:-2,1:-1,1:-1] - 2 * padded[1:-1,1:-1,1:-1] + padded[2:,1:-1,1:-1]) / hx ** 2 + \
          (padded[1:-1,:-2,1:-1] - 2 * padded[1:-1,1:-1,1:-1] + padded[1:-1,2:,1:-1]) / hy ** 2 + \
          (padded[1:-1,1:-1,:-2] - 2 * padded[1:-1,1:-1,1:-1] + padded[1:-1,1:-1,2:]) / hz ** 2
    
    out[0,:,:] = 0
    out[-1,:,:] = 0
    out[:,0,:] = 0
    out[:,-1,:] = 0
    out[:,:,0] = 0
    out[:,:,-1] = 0
    
    return out

result = np.zeros((nodes, nodes, nodes, time_ticks))
# result[:,:,:,0:1] = np.sin(np.pi * X / Lx) * np.sin(np.pi * Y / Ly) * np.sin(np.pi * Z / Lz)
# result[:,:,:,1] = result[:,:,:,0] + tau ** 2 * laplasian(result[:,:,:,0]) / 2

result[:,:,:,0] = analytical[:,:,:,0]
result[:,:,:,1] = analytical[:,:,:,1]

np.set_printoptions(precision=3)
np.set_printoptions(floatmode='fixed')
# print(analytical[:,:,:,1])

for i in range(2, time_ticks):
    result[:,:,:,i] = 2 * result[:,:,:,i-1] - result[:,:,:,i-2] + tau ** 2 * laplasian(result[:,:,:,i-1]) / 2
    print(np.max(np.abs(analytical[:,:,:,i] - result[:,:,:,i])))


5.05887433055463e-08
1.517662160388511e-07
3.035323926647848e-07
5.058872357688315e-07
7.588306966122005e-07
1.0623627144656922e-06
1.41648321716481e-06
1.8211921206656712e-06
2.2764893296001176e-06
2.7823747364985607e-06
3.338848222345092e-06
3.945909656799529e-06
4.603558897420257e-06
5.311795790330365e-06
6.0706201699956e-06
6.880031859113345e-06
7.740030668723641e-06
8.65061639832021e-06
